# Init

In [1]:
import os, sys

ROOT_DIR = '../..'
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

from dotenv import load_dotenv
load_dotenv('.env')

DATAPATH = os.getenv("DATAPATH")

import pandas as pd 
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error

In [2]:
df = pd.read_parquet(os.path.join(DATAPATH, 'staged', 'preprocessed.parquet'))
df

,id_project,tipologia,area_terreno,area_construida,area_fundacao,area_fachada,area_parede,qtde_bwcs,preco_material_total,preco_execucao_total
0,0,0,0.925,0.000000,0.352602,0.337960,0.424658,0.5,673.53120,378.19696
1,1,1,0.000,0.014633,0.000000,1.000000,1.000000,0.0,548.42627,308.54105
2,2,1,1.000,1.000000,1.000000,0.000000,0.000000,1.0,1030.66774,492.34065
3,3,1,-0.250,0.058138,0.275927,-0.406031,-0.429514,1.0,NaN,NaN


In [3]:
def train_regression(X, Y):
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    return model


def predict(value, params):
    b, a = params
    return b + a * value


def show_results(Y, Y_pred):
    df_preds = pd.DataFrame({
        'y_true': Y,
        'y_pred': Y_pred,
    })
    df_preds['diff'] = (df_preds['y_true'] - df_preds['y_pred']) * -1
    display(df_preds)

In [4]:
train_mask = df['id_project'] != 3
test_mask = df['id_project'] == 3
display(df[train_mask])
display(df[test_mask])

,id_project,tipologia,area_terreno,area_construida,area_fundacao,area_fachada,area_parede,qtde_bwcs,preco_material_total,preco_execucao_total
0,0,0,0.925,0.000000,0.352602,0.33796,0.424658,0.5,673.53120,378.19696
1,1,1,0.000,0.014633,0.000000,1.00000,1.000000,0.0,548.42627,308.54105
2,2,1,1.000,1.000000,1.000000,0.00000,0.000000,1.0,1030.66774,492.34065


,id_project,tipologia,area_terreno,area_construida,area_fundacao,area_fachada,area_parede,qtde_bwcs,preco_material_total,preco_execucao_total
3,3,1,-0.25,0.058138,0.275927,-0.406031,-0.429514,1.0,NaN,NaN


# Model Material

In [5]:
X = df.loc[train_mask, 'area_fundacao']
Y = df.loc[train_mask,'preco_material_total']
model_material = train_regression(X,Y)
print(model_material.summary())

                             OLS Regression Results                             
Dep. Variable:     preco_material_total   R-squared:                       0.990
Model:                              OLS   Adj. R-squared:                  0.979
Method:                   Least Squares   F-statistic:                     94.75
Date:                  Tue, 14 Nov 2023   Prob (F-statistic):             0.0652
Time:                          22:11:18   Log-Likelihood:                -13.373
No. Observations:                     3   AIC:                             30.75
Df Residuals:                         1   BIC:                             28.94
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           529.5786  

/home/dockeruser/.local/lib/python3.11/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [6]:
Y_pred = [predict(x, model_material.params) for x in X]
show_results(Y, Y_pred)

,y_true,y_pred,diff
0,673.53120,702.644088,29.112888
1,548.42627,529.578643,-18.847627
2,1030.66774,1020.402479,-10.265261


In [7]:
mae_material = mean_absolute_error(Y, Y_pred)
mae_material

19.408591887683542

## Nota

* Ajuste do modelo de regressão realizado com uma amostra de tamanho n = 3 e uma única variável independente (`area_fundacao`)
* O valor $R^2$ apresenta que as variáveis independentes explicam 99% da variação na variável dependente
* Os valores p-value para os coeficientes ajustados apresentam significância estatística, apesar do valor ligeiramente acima dos 0.05 para o coeficiente associado à variável `area_fundacao`
* Para os dados da amostra, o Erro Médio Absoluto é de aproximadamente 19.41
* Apesar do ajuste satisfatório o intervalo de confiança para os coeficientes ajustados é muito amplo, o que era esperado dado uma amostra tão pequena (3 exemplos).

# Model Execução

In [8]:
X = df.loc[train_mask, 'area_fundacao']
Y = df.loc[train_mask,'preco_execucao_total']
model_execution = train_regression(X,Y)
print(model_execution.summary())

                             OLS Regression Results                             
Dep. Variable:     preco_execucao_total   R-squared:                       0.999
Model:                              OLS   Adj. R-squared:                  0.998
Method:                   Least Squares   F-statistic:                     1130.
Date:                  Tue, 14 Nov 2023   Prob (F-statistic):             0.0189
Time:                          22:11:18   Log-Likelihood:                -6.6934
No. Observations:                     3   AIC:                             17.39
Df Residuals:                         1   BIC:                             15.58
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           310.5745  

/home/dockeruser/.local/lib/python3.11/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [9]:
Y_pred = [predict(x, model_execution.params) for x in X]
show_results(Y, Y_pred)

,y_true,y_pred,diff
0,378.19696,375.056072,-3.140888
1,308.54105,310.574455,2.033405
2,492.34065,493.448133,1.107483


In [10]:
mae_execution = mean_absolute_error(Y, Y_pred)
mae_execution

2.093925547591747

## Nota

* Ajuste do modelo de regressão realizado com uma amostra de tamanho n = 3 e uma única variável independente (`area_fundacao`)
* O valor $R^2$ apresenta que as variáveis independentes explicam 99,9% da variação na variável dependente
* Os valores p-value para os coeficientes ajustados apresentam significância estatística
* Para os dados da amostra, o Erro Médio Absoluto é de aproximadamente 2.1
* Apesar do ajuste satisfatório o intervalo de confiança para os coeficientes ajustados é amplo, o que era esperado dado uma amostra tão pequena (3 exemplos).

# Predição para o projeto 4

In [11]:
X = df.loc[test_mask, 'area_fundacao']
Y_pred_material = [predict(x, model_material.params) for x in X]
Y_pred_execution = [predict(x, model_execution.params) for x in X]
budget = np.sum([Y_pred_material, Y_pred_execution]) * 1000
print(f'Predicted budget: {budget:>.2f}')
print(f'MAE: {((mae_material + mae_execution) * 1000):>.2f}')

Predicted budget: 1026044.63
MAE: 21502.52


# Nota

* Os parâmetros ajustados foram empregados para estimar o orçamento do projeto 4
* O resultado é multiplicado por 1000 devido à transformação realizada no pré-processamento dos dados
* O valor estimado aproximado para o orçamento do projeto 4 é de R$ 1.026.044,63 reais podendo variar aproximadamente 21.502,52 reais para mais ou para menos.